### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

import warnings
warnings.simplefilter("ignore")

pd.set_option("display.max_columns", 999)

___

### Constants

In [2]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

N_PREDICT = 50 
TOP_K_RECALL = 50
TOP_K_PRECISION = 5

VAL_MATCHER_WEEKS = 9
VAL_RANKER_WEEKS = 3

### Functions

In [3]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(60 * '=')
    print(f"Shape: {df_data.shape}\tUsers: {df_data[USER_COL].nunique()}\t"
          f"Items: {df_data[ITEM_COL].nunique()}\n")

In [4]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, \
        df_data.apply(lambda row: recall_at_k(row[col_name], 
                                              row[ACTUAL_COL], 
                                              k=top_k), axis=1).mean()

In [5]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, \
        df_data.apply(lambda row: precision_at_k(row[col_name], 
                                                 row[ACTUAL_COL], 
                                                 k=top_k), axis=1).mean()

___

In [6]:
data = pd.read_csv('./data/retail_train.csv')
item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

In [7]:
# Переведем названия столбцов в малые буквы
item_features.columns = list(map(str.lower, item_features.columns))
user_features.columns = list(map(str.lower, user_features.columns))

# переименуем столбцы
item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

### Split dataset

In [8]:
# Максимальное значение недели
data['week_no'].max()

95

In [9]:
# берем данные для тренировки matching модели (модель, выдающая список рекомендаций)
# данные старше 9 недель
data_train_matcher = data[data['week_no'] < data['week_no'].max() - VAL_MATCHER_WEEKS]

# берем данные для валидации matching модели
# данные от 9 до 3 недель назад (точнее до последней недели датасета)
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - VAL_MATCHER_WEEKS) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели (модель, ранжирующая выдачу предыдущей модели)
data_train_ranker = data_val_matcher.copy()
# берем данные для теста ranking, matching модели
# данные за последние 3 недели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [10]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12)	Users: 2498	Items: 83685

val_matcher
Shape: (169711, 12)	Users: 2154	Items: 27649

train_ranker
Shape: (169711, 12)	Users: 2154	Items: 27649

val_ranker
Shape: (118314, 12)	Users: 2042	Items: 24329



### Prefilter

In [11]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, 
                                     item_features=item_features, 
                                     take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [12]:
# ищем общих пользователей
common_users = data_train_matcher['user_id'].values

data_val_matcher = data_val_matcher[data_val_matcher['user_id'].isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker['user_id'].isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker['user_id'].isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13)	Users: 2495	Items: 5001

val_matcher
Shape: (169615, 12)	Users: 2151	Items: 27644

train_ranker
Shape: (169615, 12)	Users: 2151	Items: 27644

val_ranker
Shape: (118282, 12)	Users: 2040	Items: 24325



### recommender train

In [13]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [80]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]

In [81]:
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


%%time
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

sorted(calc_recall(result_eval_matcher, TOP_K_RECALL), key=lambda x: x[1],reverse=True)

sorted(calc_precision(result_eval_matcher, TOP_K_PRECISION), key=lambda x: x[1],reverse=True)

In [82]:
test_k = [20, 50, 100, 200, 500]

In [83]:
%%time
for k in test_k:
    result_eval_matcher[f'own_rec_{k}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=k))
    result_eval_matcher[f'sim_item_rec_{k}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=k))
    result_eval_matcher[f'als_rec_{k}'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=k))

CPU times: user 10min 21s, sys: 26.7 s, total: 10min 47s
Wall time: 3min 16s


In [101]:
result_eval_matcher.head(2)

,user_id,actual,own_rec_20,sim_item_rec_20,als_rec_20,own_rec_50,sim_item_rec_50,als_rec_50,own_rec_100,sim_item_rec_100,als_rec_100,own_rec_200,sim_item_rec_200,als_rec_200,own_rec_500,sim_item_rec_500,als_rec_500
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 1007512, 826120, 5577022, 907002, 983...","[1037332, 1047619, 908312, 841584, 1094924, 85...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 1007512, 826120, 5577022, 907002, 983...","[1037332, 1047619, 908312, 841584, 1094924, 85...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 1007512, 826120, 5577022, 907002, 983...","[1037332, 1047619, 908312, 841584, 1094924, 85...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 1007512, 826120, 5577022, 907002, 983...","[1037332, 1047619, 908312, 841584, 1094924, 85...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 1007512, 826120, 5577022, 907002, 983...","[1037332, 1047619, 908312, 841584, 1094924, 85..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 933067, 985999, 880888, 819...","[5569230, 916122, 1029743, 951197, 844179, 865...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 933067, 985999, 880888, 819...","[5569230, 916122, 1029743, 951197, 844179, 865...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 933067, 985999, 880888, 819...","[5569230, 916122, 1029743, 951197, 844179, 865...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 933067, 985999, 880888, 819...","[5569230, 916122, 1029743, 951197, 844179, 865...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1137346, 5569845, 933067, 985999, 880888, 819...","[5569230, 916122, 1029743, 951197, 844179, 865..."


In [92]:
for k in test_k:
    print(sorted(calc_recall(result_eval_matcher, k), key=lambda x: x[1],reverse=True)[0])

('own_rec_20', 0.03928427679372909)
('own_rec_50', 0.06525657038145175)
('own_rec_100', 0.09604492955885034)
('own_rec_200', 0.13537278412833242)
('own_rec_500', 0.18205324555508678)


Очевидно, что при увеличении k значение метрики recall также увеличивается. 

___

## Ranking

### подготовка train датасета

In [20]:
%%time
# создаем датафрейм со всеми уникальными юзерами из тренировочного
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique(), columns=[USER_COL])
# для каждого делаем предсказания get_own_recommendation (ItemItem)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 9.06 s, sys: 10.2 ms, total: 9.07 s
Wall time: 9.1 s


In [21]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [22]:
# разворачиваем все это в датафрейм, где в одной строке один юзер и один итем
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), 
                                     axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [23]:
df_match_candidates.head(5)

,user_id,item_id
0,2070,1105426
0,2070,1097350
0,2070,879194
0,2070,948640
0,2070,928263


In [24]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (107550, 2)	Users: 2151	Items: 4574



In [25]:
# создадим датафрейм из тренировочного, и так как товары здесь были реально куплены проставим
# в таргете 1
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1

In [26]:
df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


In [27]:
# для нулей мерджим с нашим подготовленным датасетом предсказанных кандидатов
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

# заполняем nan нулями
df_ranker_train['target'].fillna(0, inplace= True)

In [28]:
df_ranker_train['target'].value_counts(normalize=True)

0.0    0.92713
1.0    0.07287
Name: target, dtype: float64

Задумывалось, как на каждого юзера по 50(N_PREDICT) кандидатов

In [29]:
df_ranker_train['user_id'].nunique() * N_PREDICT, df_ranker_train.shape[0]

(107550, 106972)

___

### Подготовка фичей

In [30]:
# намерджим к нашему датасету фичей из user и item features датасетов
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

In [31]:
df_ranker_train.head(3)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [32]:
# Для создания фичей смерджим датафрейм с покупками и с данными по товарам
data_item_df = pd.merge(data, item_features, on='item_id', how='left')

### Фичи user_id:

In [33]:
# средний чек каждого юзера по корзинам
mean_basket = data_item_df.groupby(['user_id', 'basket_id'])['sales_value'].mean().reset_index().groupby(['user_id'])['sales_value'].mean().reset_index()
mean_basket.rename(columns={'sales_value': 'mean_basket'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(mean_basket, on='user_id', how='left')

In [34]:
# средний чек юзера по каждой из категорий
mean_commodity = data_item_df.groupby(['user_id', 'commodity_desc'])['sales_value'].mean().reset_index()
mean_commodity.rename(columns={'sales_value': 'mean_commodity'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(mean_commodity, on=['user_id', 'commodity_desc'], how='left')

In [35]:
# количество покупок на каждую категорию
quantity_commodity = data_item_df.groupby(['user_id', 'commodity_desc'])['quantity'].count().reset_index()
quantity_commodity.rename(columns={'quantity': 'quantity_commodity'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(quantity_commodity, on=['user_id', 'commodity_desc'], how='left')

In [36]:
# Частотность покупок раз/месяц
# для начала разобьем наши дни на месяцы (несколько приблизительно, по 30 дней)
data_item_df['month'] = data_item_df['day'] // 30

In [37]:
month_quantity = data_item_df.groupby(['user_id', 'month'])['quantity'].count().reset_index()
month_quantity.rename(columns={'quantity': 'month_quantity'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(month_quantity, on=['user_id', 'month'], how='left')

In [38]:
# Доля покупок в выходные

# так как датасет от X5(а значит скорее всего собран по России),  
# предполагаем, что выходные - это 6 и 7 день недели 
# Первая неделя длиной 5 дней, значит она началась со среды. Смещение составит 2 и 3 дня 
# для воскресенья и субботы сооответственно

# создадим функцию определения субботы/воскресенья в зависимости от дня/недели
weekends = lambda day, week: (day == week * 7 - 2) or (day == week * 7 - 3)

data_item_df['weekend_buy'] = data_item_df.apply(lambda x: weekends(x['day'], x['week_no']) * 1, 
                                                 axis=1)

In [39]:
# Долю покупок утром/днем/вечером

# с 0 до 6 часов — ночь 
# с 6 до 12 часов — утро 
# с 12 до 18 часов — день 
# с 18 до 24 часов — вечер

def day_time(trans_time):
    if 0 <= trans_time < 600:
        return 'night'
    elif 600 <= trans_time < 1200:
        return 'morning'
    elif 1200 <= trans_time < 1800:
        return 'day'
    elif 1800 <= trans_time <= 2359:
        return 'evening'
    else:
        return None

In [40]:
data_item_df['day_time'] = data_item_df.apply(lambda x: day_time(x['trans_time']), axis=1)

### Фичи item_id:

In [41]:
# среднее количество покупок товара в неделю
item_week_quantity = data_item_df.groupby(['item_id', 'week_no'])['quantity'].count().reset_index().groupby('item_id')['quantity'].mean().reset_index()
item_week_quantity.rename(columns={'quantity': 'item_week_quantity'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(item_week_quantity, on='item_id', how='left')

In [42]:
# Среднее кол-во покупок 1 товара в категории в неделю
item_commodity_quantity = data_item_df.groupby(['item_id', 'commodity_desc', 'week_no'])['quantity'].mean().reset_index().groupby(['item_id', 'commodity_desc'])['quantity'].mean().reset_index()
item_commodity_quantity.rename(columns={'quantity': 'item_commodity_quantity'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(item_commodity_quantity, on=['item_id', 'commodity_desc'], how='left')

In [43]:
# (Кол-во покупок в неделю) / (Среднее кол-во покупок 1 товара в категории в неделю)

data_item_df['week-commodity'] = data_item_df['item_week_quantity'] / data_item_df['item_commodity_quantity']

In [44]:
# цена
data_item_df['price'] = data_item_df['sales_value'] / data_item_df['quantity']

### Фичи пары user_id - item_id

In [56]:
user_com_week = data_item_df.groupby(['user_id', 'commodity_desc', 'week_no'])['quantity'].count().reset_index()
user_com_week.rename(columns={'quantity': 'user_com_week'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(user_com_week, on=['user_id', 'commodity_desc', 'week_no'], how='left')

In [58]:
item_com_week = data_item_df.groupby(['commodity_desc', 'week_no'])['quantity'].count().reset_index()
item_com_week.rename(columns={'quantity': 'item_com_week'}, inplace=True)

# мерджим с фичевым датасетом
data_item_df = data_item_df.merge(item_com_week, on=['commodity_desc', 'week_no'], how='left')

In [60]:
# (Кол-во покупок юзером конкретной категории в неделю) - 
# (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
data_item_df['user_item_diff'] = data_item_df['user_com_week'] - data_item_df['item_com_week']

In [61]:
# (Кол-во покупок юзером конкретной категории в неделю) / 
# (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
data_item_df['user_item_diff'] = data_item_df['user_com_week'] / data_item_df['item_com_week']

### Присоединим наши созданные фичи к трейновому датасету

In [72]:
feat_cols = [
    'user_id',
    'item_id',
    'mean_basket',
    'mean_commodity',
    'quantity_commodity',
    'month',
    'month_quantity',
    'weekend_buy',
    'day_time',
    'item_week_quantity',
    'item_commodity_quantity',
    'week-commodity',
    'price',
    'user_com_week',
    'item_com_week',
    'user_item_diff'
]

In [77]:
# df_ranker_train = df_ranker_train.merge(data_item_df[feat_cols], on=['user_id', 'item_id'], how='left')

### Тренируем ранжирующую модель

Сначала обучим нашу модель на датасете без сгенерированных фичей

In [94]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [95]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [96]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [97]:
df_ranker_predict = df_ranker_train.copy()

In [98]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [102]:
df_ranker_predict.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,proba_item_purchase
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.036421
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,0.022789


### Предскажем на тестовом датасете

In [100]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [103]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 9 s, sys: 8.34 ms, total: 9.01 s
Wall time: 9.03 s


Без применения модели ранжирования

In [105]:
sorted(calc_precision(result_eval_ranker, TOP_K_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.1444117647058813)]

In [106]:
# функция переранжирования в зависимости от предсказанного скора
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', 
                                                                               ascending=False).head(5)['item_id'].tolist()

In [107]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [108]:
print(*sorted(calc_precision(result_eval_ranker, TOP_K_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.15331592689294912)
('own_rec', 0.1444117647058813)


### Обучим модель на датасете с новыми фичами

In [109]:
df_ranker_train = df_ranker_train.merge(data_item_df[feat_cols], on=['user_id', 'item_id'], how='left')

In [110]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [112]:
cat_feats = [
    'manufacturer',
    'department',
    'brand',
    'commodity_desc',
    'sub_commodity_desc',
    'curr_size_of_product',
    'age_desc',
    'marital_status_code',
    'income_desc',
    'homeowner_desc',
    'hh_comp_desc',
    'household_size_desc',
    'kid_category_desc',
    'weekend_buy',
    'day_time',
]

In [116]:
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [117]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)
train_preds = lgb.predict_proba(X_train)

In [118]:
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [121]:
result_eval_ranker['feat_reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [123]:
print(*sorted(calc_precision(result_eval_ranker, TOP_K_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('feat_reranked_own_rec', 0.21107049608354758)
('reranked_own_rec', 0.15331592689294912)
('own_rec', 0.1444117647058813)


За счет новых фичей получили неплохую прибавку к скору модели